# SpinGlassPEPS

W tej sekcji pokażemy zastosowanie sieci tensorowych do optymalizacji i próbkowania z modelu isinga na przykładzie algorytmu SpinGlassPEPS.

## Model Pottsa



Pewne uogólnienie modelu Isinga

$$
H(x_{\hat{N}}) = \sum_{(m, n) \in \mathcal{F}} E_{x_n} E_{x_m} + \sum_{n=1}^{\hat{N}} E_{x_{n}}.
$$



## Schemat działania algorytmu

![image](pictures/alg_software-1.png)

 Problem Isinga (a) jest przekształcany w Hamiltonian Pottsa określony na grafie króla (b). Dzięki temu funkcja podziału tego Hamiltonianu może zostać przedstawiona jako tensorowa sieć PEPS na kwadratowej siatce (c). Główny algorytm wykonuje przeszukiwanie typu branch and bound w przestrzeni prawdopodobieństw, stopniowo budując najbardziej prawdopodobne konfiguracje poprzez dodawanie po jednej zmiennej Pottsa. Warunkowe prawdopodobieństwa brzegowe uzyskiwane są przez przybliżone zwęrzanie odpowiadającej im sieci tensorowej (d). Pełny przebieg branch and bound daje w efekcie kandydata na najbardziej prawdopodobną konfigurację (stan podstawowy) (e), wraz ze zlokalizowanymi wzbudzeniami nałożonymi na nią (f).

## Zamiana dwave na hamiltonian potsa

![image](pictures/tn_dw.png)

rupujemy spiny w każdej komórce elementarnej, uzyskując uogólniony Hamiltonian Pottsa, zdefiniowany na siatce kwadratowej z oddziaływaniami między najbliższymi sąsiadami (niebieskie linie) oraz sprzężeniami po przekątnych (zielone linie), jak pokazano na panelach (b) i (d).

# WIP

* obroty
* duże komórki elementarne (krótko)
* jak działa branch and bound u nas (skrócone)


# Przykłady

## Model isinga na grafie króla

OBRAZEK

In [1]:
using SpinGlassPEPS
using Logging

disable_logging(LogLevel(1))


function get_instance(topology::NTuple{3, Int})
    m, n, t = topology
    "$(@__DIR__)/instancje/$(m)x$(n)x$(t).txt"
end


function run_square_diag_bench(::Type{T}; topology::NTuple{3, Int}) where {T}
    m, n, _ = topology
    instance = get_instance(topology)
    lattice = super_square_lattice(topology)

    best_energies = T[]

    potts_h = potts_hamiltonian(
        ising_graph(instance),
        spectrum = full_spectrum,
        cluster_assignment_rule = lattice,
    )

    params = MpsParameters{T}(; bond_dim = 16, num_sweeps = 1)
    search_params = SearchParameters(; max_states = 2^8, cutoff_prob = 1E-4)

    for transform ∈ all_lattice_transformations
        net = PEPSNetwork{KingSingleNode{GaugesEnergy}, Dense, T}(
            m, n, potts_h, transform,
        )

        ctr = MpsContractor(SVDTruncate, net, params; 
            onGPU = false, beta = T(2), graduate_truncation = true,
        )

        droplets = SingleLayerDroplets(; max_energy = 10, min_size = 5, metric = :hamming)
        merge_strategy = merge_branches(
            ctr; merge_prob = :none , droplets_encoding = droplets,
        )

        sol, _ = low_energy_spectrum(ctr, search_params, merge_strategy)
        droplets = unpack_droplets(sol, T(2))
        ig_states = decode_potts_hamiltonian_state.(Ref(potts_h), droplets.states)
        ldrop = length(droplets.states)

        println("Number of droplets for transform $(transform) is $(ldrop)")

        push!(best_energies, sol.energies[1])
        clear_memoize_cache()
    end

    ground = best_energies[1]

    println("Best energy found: $(ground)")
end


T = Float64
@time run_square_diag_bench(T; topology = (3, 3, 2))

┌ Warning: CUDA runtime library `libcublasLt.so.12` was loaded from a system path, `/usr/local/cuda-12.6/lib64/libcublasLt.so.12`.
│ This may cause errors.
│ 
│ If you're running under a profiler, this situation is expected. Otherwise,
│ ensure that your library path environment variable (e.g., `PATH` on Windows
│ or `LD_LIBRARY_PATH` on Linux) does not include CUDA library paths.
│ 
│ In any other case, please file an issue.
└ @ CUDA ~/.julia/packages/CUDA/oymHm/src/initialization.jl:218
┌ Warning: CUDA runtime library `libnvJitLink.so.12` was loaded from a system path, `/usr/local/cuda-12.6/lib64/libnvJitLink.so.12`.
│ This may cause errors.
│ 
│ If you're running under a profiler, this situation is expected. Otherwise,
│ ensure that your library path environment variable (e.g., `PATH` on Windows
│ or `LD_LIBRARY_PATH` on Linux) does not include CUDA library paths.
│ 
│ In any other case, please file an issue.
└ @ CUDA ~/.julia/packages/CUDA/oymHm/src/initialization.jl:218
┌ Warning:

Number of droplets for transform LatticeTransformation((1, 2, 3, 4), false) is 14
Number of droplets for transform LatticeTransformation((4, 1, 2, 3), true) is 16
Number of droplets for transform LatticeTransformation((3, 4, 1, 2), false) is 16
Number of droplets for transform LatticeTransformation((2, 3, 4, 1), true) is 12
Number of droplets for transform LatticeTransformation((4, 3, 2, 1), false) is 14
Number of droplets for transform LatticeTransformation((2, 1, 4, 3), false) is 12
Number of droplets for transform LatticeTransformation((1, 4, 3, 2), true) is 12
Number of droplets for transform LatticeTransformation((3, 2, 1, 4), true) is 14
Best energy found: -19.93322585045091
 59.613325 seconds (170.62 M allocations: 8.399 GiB, 2.01% gc time, 99.47% compilation time: 2% of which was recompilation)


## Mały Pegasus

In [2]:
using SpinGlassPEPS
using CUDA

onGPU = CUDA.has_cuda_gpu()

function run_pegasus_bench(::Type{T}; topology::NTuple{3, Int}) where {T}
    m, n, t = topology
    instance = "$(@__DIR__)/instances/P4_CBFM-P.txt"
    results_folder = "$(@__DIR__)/lbp"
    isdir(results_folder) || mkdir(results_folder)

    lattice = pegasus_lattice(topology)

    potts_h = potts_hamiltonian(
        ising_graph(instance),
        spectrum = full_spectrum,
        cluster_assignment_rule = lattice,
    )

    potts_h = truncate_potts_hamiltonian(potts_h, T(2), 2^16, results_folder, "P4_CBFM-P"; tol=1e-6, iter=2)

    params = MpsParameters{T}(bond_dim=16, num_sweeps=1)
    search_params = SearchParameters(max_states=2^10, cutoff_prob=1e-4)

    best_energies = T[]

    for transform in all_lattice_transformations
        net = PEPSNetwork{SquareCrossDoubleNode{GaugesEnergy}, Sparse, T}(m, n, potts_h, transform)
        ctr = MpsContractor(Zipper, net, params; onGPU=onGPU, beta=T(1), graduate_truncation=true)

        droplets = SingleLayerDroplets(max_energy=10, min_size=54, metric=:hamming)
        merge_strategy = merge_branches(ctr; merge_prob=:none, droplets_encoding=droplets)

        sol, _ = low_energy_spectrum(ctr, search_params, merge_strategy)
        sol2 = unpack_droplets(sol, T(2))

        println("Droplet energies: $(sol2.energies)")

        push!(best_energies, sol.energies[1])
        clear_memoize_cache()
    end

    ground = best_energies[1]
    @assert all(ground .≈ best_energies)
    println("Best energy found: $(best_energies[1])")
end

T = Float64
@time run_pegasus_bench(T; topology = (3, 3, 3))

LoadError: ArgumentError: Package CUDA not found in current path.
- Run `import Pkg; Pkg.add("CUDA")` to install the CUDA package.

## Wypełnianie obrazka (wymaga GPU)

In [ ]:
using SpinGlassPEPS
using CUDA
using Pkg
using Logging

# for visualisation of results, we need following packages
try
    using Colors, Images
catch e
    if isa(e, ArgumentError) || isa(e, LoadError)
        Pkg.add("Colors")
        Pkg.add("Images")
    else
        rethrow(e)
    end
end

disable_logging(LogLevel(1))

instance = "$(@__DIR__)/instances/triplepoint4-plain-ring.h5"
onGPU = CUDA.has_cuda_gpu()

GEOMETRY = SquareSingleNode
LAYOUT = GaugesEnergy
SPARSITY = Sparse
STRATEGY = SVDTruncate
GAUGE =  NoUpdate

function bench_inpaining(::Type{T}, β::Real, max_states::Integer, bond_dim::Integer) where {T}
	potts_h= potts_hamiltonian(instance, 120, 120)

	params = MpsParameters{T}(; bond_dim = bond_dim, method = :svd)
	search_params = SearchParameters(; max_states = max_states)
	net = PEPSNetwork{GEOMETRY{LAYOUT}, SPARSITY, T}(120, 120, potts_h, rotation(0))
	ctr = MpsContractor{STRATEGY, GAUGE, T}(net, params; onGPU = onGPU, beta = convert(Float64, β), graduate_truncation = true)
    droplets = SingleLayerDroplets(; max_energy = 100, min_size = 100 , metric = :hamming, mode=:RMF)
	merge_strategy = merge_branches(ctr; merge_prob = :none, droplets_encoding = droplets)

	sol, info = low_energy_spectrum(ctr, search_params, merge_strategy)
    ground = sol.energies[begin]

    println("Best energy found: $(ground)")
    sol
end

function visualize_result(sol::Solution)
    solution = sol.states[begin]
    solution = reshape(solution, (120, 120))

    sol2 = unpack_droplets(sol, 6)
    droplet_state = sol2.states[2]
    droplet_state = reshape(droplet_state, (120, 120))

    droplet = findall(solution .!= droplet_state)

    color_map = Dict(
        1 => RGB(225/255, 0.0, 100/255),
        2 => RGB(100/255, 225/255, 0.0),
        3 => RGB(0/255, 100/255, 225/255),
        4 => RGB(1.0, 1.0, 1.0)
    )

    img = zeros(RGB, 120, 120)

    for i in 1:120
        for j in 1:120
            img[j, i] = color_map[solution[i, j]]
        end
    end

    save(joinpath("$(@__DIR__)/inpaining_solution.png"), img)
    println("Solution visualisation saved to $(@__DIR__)/inpaining_solution.png")
    
    for (i,j) in Tuple.(droplet)
        img[j, i] = RGB(255/255, 255/255, 0/255)
    end

    save(joinpath("$(@__DIR__)/inpaining_droplet.png"), img)
    println("Droplet visualisation saved to $(@__DIR__)/inpaining_droplet.png")
end


sol = bench_inpaining(Float64, 6, 64, 4)
visualize_result(sol)